In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
x = np.linspace(0, 50, 501)

In [ ]:
y = np.sin(x)

In [ ]:
plt.plot(x,y)

In [ ]:
df = pd.DataFrame(data = y, index = x, columns = ['Sine'])

In [ ]:
split_ind = int(len(df) * 0.1)

In [ ]:
split_ind = int(len(df) - split_ind)
split_ind

In [ ]:
train = df.iloc[:split_ind]
test = df.iloc[split_ind:]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
length = 25
batch_size = 1

generator = TimeseriesGenerator(scaled_train, scaled_train,
                               length=length, batch_size=batch_size)

In [ ]:
a,b = generator[0]

In [ ]:
model = Sequential()

model.add(SimpleRNN(32, input_shape=(25,1), activation = 'relu', return_sequences = True ))

model.add(LSTM(32, input_shape=(25,1), activation = 'relu' ))

model.add(Dense(1))


model.compile(optimizer = 'adam', loss = 'mse')

In [ ]:
model.fit_generator(generator, epochs = 5)

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
eval_batch = scaled_train[-length:].reshape(1,25,1)

In [ ]:
model.predict(eval_batch)

In [ ]:
scaled_test[0]

In [ ]:

test_predictions = []
current_batch = scaled_train[-length:].reshape(1,25,1)

for i in range(len(scaled_test)):
    
    current_pred = model.predict(current_batch)[0]
    
    test_predictions.append(current_pred)
    
    current_batch = np.append(current_batch[:,1:,:], [[current_pred]], axis = 1)
    

test_predictions

In [ ]:
scaled_test

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)
true_predictions

In [ ]:
test['Predictions'] = true_predictions

In [ ]:
test

In [ ]:
test.plot()

In [ ]:
length = 25
early_stop = EarlyStopping(patience = 2, monitor = 'val_loss')

generator = TimeseriesGenerator(scaled_train, scaled_train,
                                    batch_size = batch_size, length = length)


validation_gen = TimeseriesGenerator(scaled_test, scaled_test,
                                    batch_size = batch_size, length = length)

In [ ]:
model = Sequential()

model.add(SimpleRNN(32, input_shape=(25,1), activation = 'relu', return_sequences = True ))

model.add(LSTM(32, input_shape=(25,1), activation = 'relu' ))

model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse')

In [ ]:
model.fit_generator(generator, 
                    validation_data = validation_gen,
                    epochs = 25, callbacks = [early_stop])

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
test_predictions = []
current_batch = scaled_train[-length:].reshape(1,49,1)

for i in range(len(scaled_test)):
    
    current_pred = model.predict(current_batch)[0]
    
    test_predictions.append(current_pred)
    
    current_batch = np.append(current_batch[:,1:,:], [[current_pred]], axis = 1)

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)
test['LSTM Predictions'] = true_predictions

In [ ]:
test.plot()